<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_?_link_to_prev_notebook.ipynb" target="_blank"><< PREV NOTEBOOK</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_?_link_to_next_notebook.ipynb" target="_blank">NEXT NOTEBOOK >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **COMPLETE WITH ANY PRE-REQUISITES (e.g. account access), IF ANY**

There are no prerequisite notebooks for this module.
- **COMPLETE WITH ANY PRE-REQUISITE NOTEBOOKS, IF ANY**
    
</div>
<hr>

# 1.1 Access Metop SST Metagranules - IFREMER Opensearch
### Data used 

| Product Description | Product Navigator | OSI SAF website link | OSI SAF Identifier |
|:--------------------:|:-------------:|:-----------------:|:-----------------:|
| Full resolution Metop-B Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-b" target="_blank">link</a>| OSI-204-b | 
| Full resolution Metop-C Sea Surface Temperature metagranules | EO:EUM:DAT:METOP:MGR-SST | <a href="https://osi-saf.eumetsat.int/products/osi-204-c" target="_blank">link</a>| OSI-204-c | 


### Learning outcomes

At the end of this notebook you will know;
* What the GHRSST format is
* What components are inside a GHRSST format file from OSI SAF

### Outline 

All OSI SAF SST are delivered in netCDF and follow the "**GHRSST format**". The Group for High Resolution Sea Surface Temperature (GHRSST) is an open international science group that promotes the application of satellites for monitoring sea surface temperature (SST) by enabling SST data producers, users and scientists to collaborate within an agreed framework of best practice. GHRSST provides a framework for SST data sharing, best practices for data processing and a forum for scientific dialog, bringing SST to the user. The GHRSST format follows specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between SST products from different agencies.


Each product includes:

* measurement data files (NetCDF-4 format)

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>

 1. [Definining satellite names](#section1)
 1. [Setting query parameters](#section2)
 1. [Importing IFREMER HTTPS credentials](#section3)
 1. [Launching an OpenSearch request](#section4)

<hr>

In [1]:
# library imports

import os
import requests         # a library that helps us acquire data across the web
import xml.dom.minidom  # a library that helps us parse XML files
import json             # a library that helps us make JSON format files
import warnings         # a library that manages Python warnings

warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Defining satellite names
[Back to top](#TOC_TOP)

</div>

This first section allows to set the satellite names in order to define a related OpenSearch query

In [2]:
# setting satellite names
SST_MetOpA = "avhrr_sst_metop_a-osisaf-l2p-v1.0"
SST_MetOpB = "avhrr_sst_metop_b-osisaf-l2p-v1.0"
SST_MetOpC = "avhrr_sst_metop_c-osisaf-l2p-v1.0"

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting query parameters
[Back to top](#TOC_TOP)

</div>

Setting Date, Time, Satellite and Lat°xLon° box.
Configure your selection

In [3]:
YYYY = "2022"
MM = "07"
DD = "09"
satellite = SST_MetOpB
lon_min = "-8"
lon_max = "3"
lat_min = "45"
lat_max = "51"

Setting connection information

<div class="alert alert-info" role="alert">

## <a id='section3'></a>3. Importing IFREMER HTTPS credentials
[Back to top](#TOC_TOP)

</div>

In order to access SST data from the IFREMER HTTPS server you will need to supply access credentials in the form of a username and password. You can apply for these once you register for an <a href="https://osi-saf.eumetsat.int/register">**OSI SAF account**</a>. We can pass these credentials to this notebook in two ways; either by creating a file called `.osisaf_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

<div class="alert alert-block alert-success">

#### Option 1: creating  `.osisaf_credentials` in your home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system. In this file we need to add the following information as follows;

```
{"ifremer_https":
    {"username": "<your_username>",
     "password": "<your_password>"
    }
}
```

You must replace `<your_username>` and `<your_password>` with the credential information you receive from OSI SAF. Make sure to save the file without any kind of extension. Once you have done this, you can read in your credentials using the commands in the following cell with `file_credentials = True`.

</div>
    
<div class="alert alert-danger" role="alert">

#### Option 2: Entering credentials directly by entering your credentials in the notebook as follows*
   
Alternatively you can enter your credentials directly in the notebook by updating the following lines with your user specific information and setting `file_credentials = False`.

```
    credentials["username"] = "<your_username>"
    credentials["password"] = "<your_password>"
```

However, while this method is convenient in the short term, it is not really recommended as you run the risk of accidentally sharing your credentials. This method also requires you to authenticate on a notebook-by-notebook basis.
    
<div>

In [4]:
# load credentials
file_credentials = True

if file_credentials == True:
    with open(os.path.join(os.path.expanduser("~"),'.osisaf_credentials')) as json_file:
        file = json.load(json_file)
        credentials = file["ifremer_https"]
else:
    # see below for more information
    credentials = {}
    credentials["username"] = "<your_username>"
    credentials["password"] = "<your_password>"

<div class="alert alert-info" role="alert">

## <a id='section4'></a>4. Launching an OpenSearch request
[Back to top](#TOC_TOP)

</div>

In [5]:
session = requests.Session()
opensearch = requests.get('https://opensearch.ifremer.fr/granules.atom?'\
                          'datasetId=' + satellite + \
                          '&startPage=0&count=1000&'\
                          'timeStart='+YYYY+'-'+MM+'-'+DD+'T00:00:00Z&'\
                          'timeEnd='+YYYY+'-'+MM+'-'+DD+'T23:59:59Z&'\
                          'geoBox='+lon_min+','+lat_min+','+lon_max+','+lat_max, verify=False)
searchresult = open('searchresult.xml','w')
searchresult.write(opensearch.text)
searchresult.close()

In [6]:
# parse granules and download files
xml_resultfile = xml.dom.minidom.parse('searchresult.xml')
link_list = xml_resultfile.getElementsByTagName('link')
granule_number = link_list.length / 3
print("%d granules" % granule_number)
for link in link_list:
    if link.getAttribute('title') == "HTTPS" :
        url = link.getAttribute('href')
        print(url)
        r = requests.get(url, auth=(credentials["username"], credentials["password"]), verify=False)
        # r.raw.decode_content = True
        filename = url.rsplit('/', 1)[1]
        print(filename)
        with open('data/' + filename, 'wb') as f:
           f.write(r.content)

7 granules
https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/190/20220709190703-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_190703-v02.0-fv01.0.nc
20220709190703-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_190703-v02.0-fv01.0.nc


FileNotFoundError: [Errno 2] No such file or directory: 'data/20220709190703-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220709_190703-v02.0-fv01.0.nc'

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_?_link_to_prev_notebook.ipynb" target="_blank"><< PREV NOTEBOOK</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_?_link_to_next_notebook.ipynb" target="_blank">NEXT NOTEBOOK >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>